In [74]:
import pandas as pd
import time
import requests
import numpy as np
import aiohttp 
import asyncio
# from Utils import scraping_essentials as se
locale.setlocale(locale.LC_ALL, 'en_US.UTF8')

'en_US.UTF8'

In [97]:
df = pd.read_csv('Transaction Status to silver__transactions+ - success_value.csv')

In [98]:
tx_status = []

In [99]:
async def get_response(session, url):
    async with session.get(url) as resp:
        text = await resp.json()
        return "Success" if text['txn']['status'] == 'Succeeded' else "Failure"

In [100]:
async with aiohttp.ClientSession() as session:
    tasks = []
    start_time = time.time()
    for tx in df['TX_HASH']:
        url = f'https://nearblocks.io/api/txn?hash={tx}'
        tasks.append(asyncio.create_task(get_response(session, url)))
    results = await asyncio.gather(*tasks)
    for result in results:
        tx_status.append(result)
    print(f"{(time.time() - start_time):.2f} seconds")

183.42 seconds


In [101]:
df['tx_status'] = tx_status

In [104]:
df['tx_status'].value_counts()

Success    9695
Failure     305
Name: tx_status, dtype: int64

In [105]:
df.to_csv("output - all txs with successValue and Failure receipts.csv",index=False)